In [1]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [48]:
def gerar_url_voo_azul(mes, dia):
  """Gera a URL de busca para uma data específica."""
  url_modelo = f"https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds=CPV&c[0].std={mes}/{dia}/2025&c[0].as=BSB&p[0].t=ADT&p[0].c=2&p[0].cp=false&f.dl=3&f.dr=3&cc=PTS"
  print(f"\nDia {dia}/{mes}")
  return url_modelo

def extrair_dados_voo(url):
  """
  Acessa a URL com Selenium e extrai informações dos voos.
  """
  # PARTE 1: Configuração do Selenium
  # Certifique-se de que o caminho para o chromedriver está correto
  service = Service()
  options = webdriver.ChromeOptions()
  # options.add_argument('--headless') # Descomente para rodar sem abrir uma janela do navegador
  driver = webdriver.Chrome(service=service, options=options)


  print(f"Acessando a página: {url}")
  driver.get(url)

  # PARTE 2: Esperar a página carregar
  # Sites de voos carregam preços dinamicamente. Precisamos esperar.
  # Este tempo de espera (15s) é um chute, pode precisar de mais ou menos.
  print("Aguardando o carregamento dos preços...")
  time.sleep(15)

  # PARTE 3: Extração dos Dados
  # Esta é a parte que você precisará adaptar.
  # Você deve usar o "Inspecionar Elemento" do seu navegador para descobrir
  # as classes, IDs ou tags dos elementos que contêm o preço e o horário.

  # Exemplo hipotético de como seria:
  try:
    # Pega todo o HTML da página depois que o JavaScript carregou tudo
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    # Encontra todos os 'cards' de voos. A classe '.flight-card' é um EXEMPLO.
    voos = soup.find_all('div', class_=re.compile('flight-card')) # <-- MUDE ESSA CLASSE
    print(voos)
    data = soup.find('div', class_="trip-info").text.strip()
    voos_list = []

    if not voos:
      print("Nenhum voo encontrado. Verifique a classe do container dos voos.")
      print("Dica: A página pode ter um popup ou aviso que impede o carregamento.")

    print(f"Encontrados {len(voos)} voos.\n")

    i = 0
    for voo in voos:
      # Exemplo para extrair horário e preço. As classes são HIPOTÉTICAS.
      # Você PRECISA inspecionar o site para encontrar as classes corretas.
      #horario_partida = voo.find('span', class_='departure-time') # <-- MUDE ESSA CLASSE
      preco_pontos = voo.find('h4', class_=re.compile('current'))     # <-- MUDE ESSA CLASSE

      if preco_pontos: # horario_partida and Horário: {horario_partida.text.strip()} |
        voos_list.append([])
        pontos = preco_pontos.text.replace(".", "").strip("pontos")
        voos_list[i].append(int(pontos))
        voos_list[i].append(data.split(" - ")[1].split("11 voo")[0])
        print(f"Preço: {preco_pontos.text.strip()} pontos")
        i += 1
    return voos_list


  except Exception as e:
    print(f"Ocorreu um erro ao extrair os dados: {e}")

  finally:
    # Fecha o navegador
    driver.quit()


Dia 05/10
Acessando a página: https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds=CPV&c[0].std=10/05/2025&c[0].as=BSB&p[0].t=ADT&p[0].c=2&p[0].cp=false&f.dl=3&f.dr=3&cc=PTS
Aguardando o carregamento dos preços...
[<div class="flight-card css-c3x279" id="QUR_MjY5Mn4gfn5DUFZ_MTAvMDUvMjAyNSAxMzo1MH5SRUN_MTAvMDUvMjAyNSAxNDozMH5_XkFEfjQ2MTJ_IH5_UkVDfjEwLzA1LzIwMjUgMTU6MzV_QlNCfjEwLzA1LzIwMjUgMTg6MTV_fg--"><div class="card flight-card__container css-6i4tbq" role="button" tabindex="0"><div aria-label="
      false
      Horário de partida 13:50.
      Horário de chegada 18:15.
      1 conexão    • 
    " class="flight-card__info left-container css-2888k0" role="button" tabindex="0"><div class="info"><div class="info-container css-1dvehwi"><p aria-label="
      false
      Horário de partida 13:50.
      Horário de chegada 18:15.
      1 conexão    • 
    " class="css-9e22zl"></p><h4 aria-hidden="true" aria-label="Horário de partida 13:50" class="departure css-qxyl5a">13:50<span class="i

In [60]:
import datetime

data_inicio = datetime.date(2025,10,5)
data_fim = datetime.date(2025,10,10)
per_dias = data_fim - data_inicio

cols = ["preco","data"]
df = pd.DataFrame(columns=cols)

for i in range(per_dias.days):
  data_inicio = data_inicio + datetime.timedelta(days=i)
  url_alvo = gerar_url_voo_azul(data_inicio.month, data_inicio.day)
  df[['preco','data']] = extrair_dados_voo(url_alvo)
  time.sleep(300)

Dia 5/10
Acessando a página: https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds=CPV&c[0].std=10/5/2025&c[0].as=BSB&p[0].t=ADT&p[0].c=2&p[0].cp=false&f.dl=3&f.dr=3&cc=PTS
Aguardando o carregamento dos preços...


KeyboardInterrupt: 